In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import math
import random

Setting environment for Gordon


In [4]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

# colors = np.random.randint(0, 255, (len(labels_index), 3))
# colors = np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')
# colors[labels_index['BackG']] = 1.

In [5]:
# build training data

sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

def compute_intersection_kernel_oneJob(i, j1):
    dist = np.minimum(train_data_normalized[i], train_data_normalized[j1:]).sum(axis=1)
    return dist

exprmt = 10

for e in xrange(exprmt):
    
    t = time.time()

    train_data = []
    train_labels = []
    train_fnames = []

    test_data = []
    test_labels = []
    test_fnames = []

    n_sample = 1000
    for name in labels:

        hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
        files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})

        n_total = hists0.shape[0]
        print (name, n_total),

        random_indices = np.random.choice(range(n_total), min(n_total, n_sample), replace=False)

        frac = int(0.8*len(random_indices))
        random_train = random_indices[:frac]
        random_test = random_indices[frac:]
        n_train = len(random_train)
        n_test = len(random_test)

        train_hists = np.c_[hists0[random_train]]
        test_hists =  np.c_[hists0[random_test]]

        train_files = files[random_train].reshape((n_train,))
        test_files = files[random_test].reshape((n_test,))

        train_data.append(train_hists)
        train_labels.append(np.ones((n_train, )) * labels_index[name])
        train_fnames.append(train_files)

        test_data.append(test_hists)
        test_labels.append(np.ones((n_test, )) * labels_index[name])
        test_fnames.append(test_files)

        print 'Train: ', n_train, 
        print 'Test: ', n_test

    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)
    train_fnames = np.concatenate(train_fnames)
    n_train = train_data.shape[0]

    test_data = np.concatenate(test_data)
    test_labels = np.concatenate(test_labels)
    test_fnames = np.concatenate(test_fnames)
    n_test = test_data.shape[0]
    
    print train_data.shape
    print train_labels.shape
    print train_fnames.shape, train_fnames.dtype
    print train_fnames[0]

    print test_data.shape
    print test_labels.shape
    print test_fnames.shape, test_fnames.dtype
    print test_fnames[0]
    
    train_data_normalized = train_data / train_data.sum(axis=1)[:,None].astype(np.float)
    train_data_normalized = np.nan_to_num(train_data_normalized)

    test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)
    test_data_normalized = np.nan_to_num(test_data_normalized)
    
    train_dist_triangle = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, i) 
                                         for i in range(n_train)))

    train_dist_mat = np.empty((n_train, n_train))
    train_dist_mat[np.triu_indices(n_train)] = train_dist_triangle
    r = np.tril_indices(n_train)
    train_dist_mat[r] = train_dist_mat.T[r]
    print 'Train kernel shape: ', train_dist_mat.shape
    
    bp.pack_ndarray_file(train_data_normalized, sift_dir+'/preprocessed/train_level0_e%d_data.bp'%e)
    bp.pack_ndarray_file(train_labels, sift_dir+'/preprocessed/train_level0_e%d_labels.bp'%e)
    bp.pack_ndarray_file(train_fnames, sift_dir+'/preprocessed/train_level0_e%d_fnames.bp'%e, chunk_size=255*4000)

    bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/test_level0_e%d_data.bp'%e)
    bp.pack_ndarray_file(test_labels, sift_dir+'/preprocessed/test_level0_e%d_labels.bp'%e)
    bp.pack_ndarray_file(test_fnames, sift_dir+'/preprocessed/test_level0_e%d_fnames.bp'%e, chunk_size=255*4000)
    
    bp.pack_ndarray_file(train_dist_mat, sift_dir+'/preprocessed/train_level0_e%d_spm_dist_mat.bp'%e)
    

    sys.stderr.write('Experiment #%d Time: %f seconds\n' % (e, (time.time()-t)))

('BackG', 5105) Train:  800 Test:  200
('5N', 1440) Train:  800 Test:  200
('7n', 3444) Train:  800 Test:  200
('7N', 2579) Train:  800 Test:  200
('12N', 1230) Train:  800 Test:  200
('Pn', 3042) Train:  800 Test:  200
('VLL', 1287) Train:  800 Test:  200
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  800 Test:  200
('Tz', 1387) Train:  800 Test:  200
('RtTg', 2639) Train:  800 Test:  200
('LRt', 1050) Train:  800 Test:  200
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  800 Test:  200
(10769, 200)
(10769,)
(10769,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0163_0028.png
(2695, 200)
(2695,)
(2695,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0309_0001.png


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:77: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:80: RuntimeWarning: invalid value encountered in divide


Train kernel shape:  (10769, 10769)
('BackG', 5105) Train:  800 Test:  200
('5N', 1440) Train:  800 Test:  200
('7n', 3444) Train:  800 Test:  200
('7N', 2579) Train:  800 Test:  200
('12N', 1230) Train:  800 Test:  200
('Pn', 3042) Train:  800 Test:  200
('VLL', 1287) Train:  800 Test:  200
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  800 Test:  200
('Tz', 1387) Train:  800 Test:  200
('RtTg', 2639) Train:  800 Test:  200
('LRt', 1050) Train:  800 Test:  200
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  800 Test:  200
(10769, 200)
(10769,)
(10769,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0205_0005.png
(2695, 200)
(2695,)
(2695,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0228_0008.png


Experiment #0 Time: 24.589878 seconds


Train kernel shape:  (10769, 10769)
('BackG', 5105) Train:  800 Test:  200
('5N', 1440) Train:  800 Test:  200
('7n', 3444) Train:  800 Test:  200
('7N', 2579) Train:  800 Test:  200
('12N', 1230) Train:  800 Test:  200
('Pn', 3042) Train:  800 Test:  200
('VLL', 1287) Train:  800 Test:  200
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  800 Test:  200
('Tz', 1387) Train:  800 Test:  200
('RtTg', 2639) Train:  800 Test:  200
('LRt', 1050) Train:  800 Test:  200
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  800 Test:  200
(10769, 200)
(10769,)
(10769,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0259_0014.png
(2695, 200)
(2695,)
(2695,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0188_0014.png


...

Experiment #7 Time: 26.564440 seconds


Train kernel shape:  (10769, 10769)
('BackG', 5105) Train:  800 Test:  200
('5N', 1440) Train:  800 Test:  200
('7n', 3444) Train:  800 Test:  200
('7N', 2579) Train:  800 Test:  200
('12N', 1230) Train:  800 Test:  200
('Pn', 3042) Train:  800 Test:  200
('VLL', 1287) Train:  800 Test:  200
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  800 Test:  200
('Tz', 1387) Train:  800 Test:  200
('RtTg', 2639) Train:  800 Test:  200
('LRt', 1050) Train:  800 Test:  200
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  800 Test:  200
(10769, 200)
(10769,)
(10769,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0192_0006.png
(2695, 200)
(2695,)
(2695,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0312_0014.png


Experiment #8 Time: 47.644950 seconds


Train kernel shape:  (10769, 10769)


Experiment #9 Time: 31.010691 seconds


In [3]:
# build training data

# sift_dir = '/oasis/projects/nsf/csd395/yuncong/Brain/learning/sift'
# sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift'
sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

def compute_intersection_kernel_oneJob(i, j1):
#     dist = .5 * np.sum(train_data_normalized + h - np.abs(train_data_normalized - h), axis=1)
    dist = np.minimum(train_data_normalized[i], train_data_normalized[j1:]).sum(axis=1)
    return dist

exprmt = 10

for e in xrange(exprmt):
    
    t = time.time()

    train_data = []
    train_labels = []
    train_fnames = []

    test_data = []
    test_labels = []
    test_fnames = []

    n_sample = 500
    for name in labels:

        hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
        hists1 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l1.bp' % {'name': name})
        hists2 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l2.bp' % {'name': name})
        files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})
    #     print (hists0.shape, hists1.shape, hists2.shape, files.shape)

        n_total = hists0.shape[0]
        print (name, n_total),

        random_indices = np.random.choice(range(n_total), min(n_total, n_sample), replace=False)

        frac = int(0.8*len(random_indices))
        random_train = random_indices[:frac]
        random_test = random_indices[frac:]
        n_train = len(random_train)
        n_test = len(random_test)

    #     train_hists = np.c_[train_hists0[random_indices], 
    #                         train_hists1[random_indices].reshape((n_train, -1)), 
    #                         train_hists2[random_indices].reshape((n_train, -1))]
        train_hists = np.c_[hists0[random_train], 
                            hists1[random_train].reshape((n_train, -1)), 
                            hists2[random_train].reshape((n_train, -1))]
        test_hists =  np.c_[hists0[random_test], 
                            hists1[random_test].reshape((n_test, -1)), 
                            hists2[random_test].reshape((n_test, -1))]

    #     train_files = train_files_[random_indices].reshape((n_train,))
        train_files = files[random_train].reshape((n_train,))
        test_files = files[random_test].reshape((n_test,))

        train_data.append(train_hists)
        train_labels.append(np.ones((n_train, )) * labels_index[name])
        train_fnames.append(train_files)

        test_data.append(test_hists)
        test_labels.append(np.ones((n_test, )) * labels_index[name])
        test_fnames.append(test_files)

        print 'Train: ', n_train, 
        print 'Test: ', n_test

    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)
    train_fnames = np.concatenate(train_fnames)
    n_train = train_data.shape[0]

    test_data = np.concatenate(test_data)
    test_labels = np.concatenate(test_labels)
    test_fnames = np.concatenate(test_fnames)
    n_test = test_data.shape[0]
    
    print train_data.shape
    print train_labels.shape
    print train_fnames.shape, train_fnames.dtype
    print train_fnames[0]

    print test_data.shape
    print test_labels.shape
    print test_fnames.shape, test_fnames.dtype
    print test_fnames[0]
    
    train_data_normalized = train_data / train_data.sum(axis=1)[:,None].astype(np.float)
    train_data_normalized = np.nan_to_num(train_data_normalized)

    test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)
    test_data_normalized = np.nan_to_num(test_data_normalized)
    
    train_dist_triangle = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, i) 
                                         for i in range(n_train)))

    train_dist_mat = np.empty((n_train, n_train))
    train_dist_mat[np.triu_indices(n_train)] = train_dist_triangle
    r = np.tril_indices(n_train)
    train_dist_mat[r] = train_dist_mat.T[r]
    print 'Train kernel shape: ', train_dist_mat.shape
    
    bp.pack_ndarray_file(train_data_normalized, sift_dir+'/preprocessed/train_level2_e%d_data.bp'%e)
    bp.pack_ndarray_file(train_labels, sift_dir+'/preprocessed/train_level2_e%d_labels.bp'%e)
    bp.pack_ndarray_file(train_fnames, sift_dir+'/preprocessed/train_level2_e%d_fnames.bp'%e, chunk_size=255*4000)

    bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/test_level2_e%d_data.bp'%e)
    bp.pack_ndarray_file(test_labels, sift_dir+'/preprocessed/test_level2_e%d_labels.bp'%e)
    bp.pack_ndarray_file(test_fnames, sift_dir+'/preprocessed/test_level2_e%d_fnames.bp'%e, chunk_size=255*4000)
    
    bp.pack_ndarray_file(train_dist_mat, sift_dir+'/preprocessed/train_level2_e%d_spm_dist_mat.bp'%e)
    

    sys.stderr.write('Experiment #%d Time: %f seconds\n' % (e, (time.time()-t)))

('BackG', 5105) Train:  400 Test:  100
('5N', 1440) Train:  400 Test:  100
('7n', 3444) Train:  400 Test:  100
('7N', 2579) Train:  400 Test:  100
('12N', 1230) Train:  400 Test:  100
('Pn', 3042) Train:  400 Test:  100
('VLL', 1287) Train:  400 Test:  100
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  400 Test:  100
('Tz', 1387) Train:  400 Test:  100
('RtTg', 2639) Train:  400 Test:  100
('LRt', 1050) Train:  400 Test:  100
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  400 Test:  100
(5969, 4200)

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:91: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:94: RuntimeWarning: invalid value encountered in divide



(5969,)
(5969,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0179_0016.png
(1495, 4200)
(1495,)
(1495,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0165_0025.png
Train kernel shape:  (5969, 5969)
('BackG', 5105) Train:  400 Test:  100
('5N', 1440) Train:  400 Test:  100
('7n', 3444) Train:  400 Test:  100
('7N', 2579) Train:  400 Test:  100
('12N', 1230) Train:  400 Test:  100
('Pn', 3042) Train:  400 Test:  100
('VLL', 1287) Train:  400 Test:  100
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  400 Test:  100
('Tz', 1387) Train:  400 Test:  100
('RtTg', 2639) Train:  400 Test:  100
('LRt', 1050) Train:  400 Test:  100
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  400 Test:  100
(5969, 4200)
(5969,)
(5969,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0176_0002.png
(1495

Experiment #0 Time: 692.485794 seconds


Train kernel shape:  (5969, 5969)
('BackG', 5105) Train:  400 Test:  100
('5N', 1440) Train:  400 Test:  100
('7n', 3444) Train:  400 Test:  100
('7N', 2579) Train:  400 Test:  100
('12N', 1230) Train:  400 Test:  100
('Pn', 3042) Train:  400 Test:  100
('VLL', 1287) Train:  400 Test:  100
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  400 Test:  100
('Tz', 1387) Train:  400 Test:  100
('RtTg', 2639) Train:  400 Test:  100
('LRt', 1050) Train:  400 Test:  100
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  400 Test:  100
(5969, 4200)
(5969,)
(5969,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0152_0005.png
(1495, 4200)
(1495,)
(1495,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0255_0017.png


...

Experiment #7 Time: 49.591066 seconds


Train kernel shape:  (5969, 5969)
('BackG', 5105) Train:  400 Test:  100
('5N', 1440) Train:  400 Test:  100
('7n', 3444) Train:  400 Test:  100
('7N', 2579) Train:  400 Test:  100
('12N', 1230) Train:  400 Test:  100
('Pn', 3042) Train:  400 Test:  100
('VLL', 1287) Train:  400 Test:  100
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  400 Test:  100
('Tz', 1387) Train:  400 Test:  100
('RtTg', 2639) Train:  400 Test:  100
('LRt', 1050) Train:  400 Test:  100
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  400 Test:  100
(5969, 4200)
(5969,)
(5969,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0290_0002.png
(1495, 4200)
(1495,)
(1495,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0261_0024.png


Experiment #8 Time: 49.218262 seconds


Train kernel shape:  (5969, 5969)


Experiment #9 Time: 50.908342 seconds


In [4]:
# build training data

sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

def compute_intersection_kernel_oneJob(i, j1):
    dist = np.minimum(train_data_normalized[i], train_data_normalized[j1:]).sum(axis=1)
    return dist

exprmt = 10

for e in xrange(exprmt):
    
    t = time.time()

    train_data = []
    train_labels = []
    train_fnames = []

    test_data = []
    test_labels = []
    test_fnames = []

    n_sample = 500
    for name in labels:

        hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
        files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})

        n_total = hists0.shape[0]
        print (name, n_total),

        random_indices = np.random.choice(range(n_total), min(n_total, n_sample), replace=False)

        frac = int(0.6*len(random_indices))
        random_train = random_indices[:frac]
        random_test = random_indices[frac:]
        n_train = len(random_train)
        n_test = len(random_test)

        train_hists = np.c_[hists0[random_train]]
        test_hists =  np.c_[hists0[random_test]]

        train_files = files[random_train].reshape((n_train,))
        test_files = files[random_test].reshape((n_test,))

        train_data.append(train_hists)
        train_labels.append(np.ones((n_train, )) * labels_index[name])
        train_fnames.append(train_files)

        test_data.append(test_hists)
        test_labels.append(np.ones((n_test, )) * labels_index[name])
        test_fnames.append(test_files)

        print 'Train: ', n_train, 
        print 'Test: ', n_test

    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)
    train_fnames = np.concatenate(train_fnames)
    n_train = train_data.shape[0]

    test_data = np.concatenate(test_data)
    test_labels = np.concatenate(test_labels)
    test_fnames = np.concatenate(test_fnames)
    n_test = test_data.shape[0]
    
    print train_data.shape
    print train_labels.shape
    print train_fnames.shape, train_fnames.dtype
    print train_fnames[0]

    print test_data.shape
    print test_labels.shape
    print test_fnames.shape, test_fnames.dtype
    print test_fnames[0]
    
    train_data_normalized = train_data / train_data.sum(axis=1)[:,None].astype(np.float)
    train_data_normalized = np.nan_to_num(train_data_normalized)

    test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)
    test_data_normalized = np.nan_to_num(test_data_normalized)
    
    train_dist_triangle = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, i) 
                                         for i in range(n_train)))

    train_dist_mat = np.empty((n_train, n_train))
    train_dist_mat[np.triu_indices(n_train)] = train_dist_triangle
    r = np.tril_indices(n_train)
    train_dist_mat[r] = train_dist_mat.T[r]
    print 'Train kernel shape: ', train_dist_mat.shape
    
    bp.pack_ndarray_file(train_data_normalized, sift_dir+'/preprocessed/train_level0_64_e%d_data.bp'%e)
    bp.pack_ndarray_file(train_labels, sift_dir+'/preprocessed/train_level0_64_e%d_labels.bp'%e)
    bp.pack_ndarray_file(train_fnames, sift_dir+'/preprocessed/train_level0_64_e%d_fnames.bp'%e, chunk_size=255*4000)

    bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/test_level0_64_e%d_data.bp'%e)
    bp.pack_ndarray_file(test_labels, sift_dir+'/preprocessed/test_level0_64_e%d_labels.bp'%e)
    bp.pack_ndarray_file(test_fnames, sift_dir+'/preprocessed/test_level0_64_e%d_fnames.bp'%e, chunk_size=255*4000)
    
    bp.pack_ndarray_file(train_dist_mat, sift_dir+'/preprocessed/train_level0_64_e%d_spm_dist_mat.bp'%e)
    

    sys.stderr.write('Experiment #%d Time: %f seconds\n' % (e, (time.time()-t)))

('BackG', 5105) Train:  300 Test:  200
('5N', 1440) Train:  300 Test:  200
('7n', 3444) Train:  300 Test:  200
('7N', 2579) Train:  300 Test:  200
('12N', 1230) Train:  300 Test:  200
('Pn', 3042) Train:  300 Test:  200
('VLL', 1287) Train:  300 Test:  200
('6N', 154) Train:  92 Test:  62
('Amb', 346) Train:  207 Test:  139
('R', 1082) Train:  300 Test:  200
('Tz', 1387) Train:  300 Test:  200
('RtTg', 2639) Train:  300 Test:  200
('LRt', 1050) Train:  300 Test:  200
('LC', 481) Train:  288 Test:  193
('AP', 483) Train:  289 Test:  194
('sp5', 3240) Train:  300 Test:  200
(4476, 200)
(4476,)
(4476,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0160_0034.png
(2988, 200)
(2988,)
(2988,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0157_0023.png


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:77: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:80: RuntimeWarning: invalid value encountered in divide


Train kernel shape:  (4476, 4476)
('BackG', 5105) Train:  300 Test:  200
('5N', 1440) Train:  300 Test:  200
('7n', 3444) Train:  300 Test:  200
('7N', 2579) Train:  300 Test:  200
('12N', 1230) Train:  300 Test:  200
('Pn', 3042) Train:  300 Test:  200
('VLL', 1287) Train:  300 Test:  200
('6N', 154) Train:  92 Test:  62
('Amb', 346) Train:  207 Test:  139
('R', 1082) Train:  300 Test:  200
('Tz', 1387) Train:  300 Test:  200
('RtTg', 2639) Train:  300 Test:  200
('LRt', 1050) Train:  300 Test:  200
('LC', 481) Train:  288 Test:  193
('AP', 483) Train:  289 Test:  194
('sp5', 3240) Train:  300 Test:  200
(4476, 200)
(4476,)
(4476,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0180_0029.png
(2988, 200)
(2988,)
(2988,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0308_0011.png


Experiment #0 Time: 3.549414 seconds


Train kernel shape:  (4476, 4476)
('BackG', 5105) Train:  300 Test:  200
('5N', 1440) Train:  300 Test:  200
('7n', 3444) Train:  300 Test:  200
('7N', 2579) Train:  300 Test:  200
('12N', 1230) Train:  300 Test:  200
('Pn', 3042) Train:  300 Test:  200
('VLL', 1287) Train:  300 Test:  200
('6N', 154) Train:  92 Test:  62
('Amb', 346) Train:  207 Test:  139
('R', 1082) Train:  300 Test:  200
('Tz', 1387) Train:  300 Test:  200
('RtTg', 2639) Train:  300 Test:  200
('LRt', 1050) Train:  300 Test:  200
('LC', 481) Train:  288 Test:  193
('AP', 483) Train:  289 Test:  194
('sp5', 3240) Train:  300 Test:  200
(4476, 200)
(4476,)
(4476,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0255_0011.png
(2988, 200)
(2988,)
(2988,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0177_0011.png


...

Experiment #7 Time: 3.432793 seconds


Train kernel shape:  (4476, 4476)
('BackG', 5105) Train:  300 Test:  200
('5N', 1440) Train:  300 Test:  200
('7n', 3444) Train:  300 Test:  200
('7N', 2579) Train:  300 Test:  200
('12N', 1230) Train:  300 Test:  200
('Pn', 3042) Train:  300 Test:  200
('VLL', 1287) Train:  300 Test:  200
('6N', 154) Train:  92 Test:  62
('Amb', 346) Train:  207 Test:  139
('R', 1082) Train:  300 Test:  200
('Tz', 1387) Train:  300 Test:  200
('RtTg', 2639) Train:  300 Test:  200
('LRt', 1050) Train:  300 Test:  200
('LC', 481) Train:  288 Test:  193
('AP', 483) Train:  289 Test:  194
('sp5', 3240) Train:  300 Test:  200
(4476, 200)
(4476,)
(4476,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0285_0020.png
(2988, 200)
(2988,)
(2988,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0294_0014.png


Experiment #8 Time: 3.213973 seconds


Train kernel shape:  (4476, 4476)


Experiment #9 Time: 3.457861 seconds


In [7]:
# build training data

sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

def compute_intersection_kernel_oneJob(i, j1):
    dist = np.minimum(train_data_normalized[i], train_data_normalized[j1:]).sum(axis=1)
    return dist

exprmt = 10

for e in xrange(exprmt):
    
    t = time.time()

    train_data = []
    train_labels = []
    train_fnames = []

    test_data = []
    test_labels = []
    test_fnames = []

    n_sample = 500
    for name in labels:

        hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
        files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})

        n_total = hists0.shape[0]
        print (name, n_total),

        random_indices = np.random.choice(range(n_total), min(n_total, n_sample), replace=False)

        frac = int(0.5*len(random_indices))
        random_train = random_indices[:frac]
        random_test = random_indices[frac:]
        n_train = len(random_train)
        n_test = len(random_test)

        train_hists = np.c_[hists0[random_train]]
        test_hists =  np.c_[hists0[random_test]]

        train_files = files[random_train].reshape((n_train,))
        test_files = files[random_test].reshape((n_test,))

        train_data.append(train_hists)
        train_labels.append(np.ones((n_train, )) * labels_index[name])
        train_fnames.append(train_files)

        test_data.append(test_hists)
        test_labels.append(np.ones((n_test, )) * labels_index[name])
        test_fnames.append(test_files)

        print 'Train: ', n_train, 
        print 'Test: ', n_test

    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)
    train_fnames = np.concatenate(train_fnames)
    n_train = train_data.shape[0]

    test_data = np.concatenate(test_data)
    test_labels = np.concatenate(test_labels)
    test_fnames = np.concatenate(test_fnames)
    n_test = test_data.shape[0]
    
    print train_data.shape
    print train_labels.shape
    print train_fnames.shape, train_fnames.dtype
    print train_fnames[0]

    print test_data.shape
    print test_labels.shape
    print test_fnames.shape, test_fnames.dtype
    print test_fnames[0]
    
    train_data_normalized = train_data / train_data.sum(axis=1)[:,None].astype(np.float)
    train_data_normalized = np.nan_to_num(train_data_normalized)

    test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)
    test_data_normalized = np.nan_to_num(test_data_normalized)
    
    train_dist_triangle = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, i) 
                                         for i in range(n_train)))

    train_dist_mat = np.empty((n_train, n_train))
    train_dist_mat[np.triu_indices(n_train)] = train_dist_triangle
    r = np.tril_indices(n_train)
    train_dist_mat[r] = train_dist_mat.T[r]
    print 'Train kernel shape: ', train_dist_mat.shape
    
    bp.pack_ndarray_file(train_data_normalized, sift_dir+'/preprocessed/train_level0_55_e%d_data.bp'%e)
    bp.pack_ndarray_file(train_labels, sift_dir+'/preprocessed/train_level0_55_e%d_labels.bp'%e)
    bp.pack_ndarray_file(train_fnames, sift_dir+'/preprocessed/train_level0_55_e%d_fnames.bp'%e, chunk_size=255*4000)

    bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/test_level0_55_e%d_data.bp'%e)
    bp.pack_ndarray_file(test_labels, sift_dir+'/preprocessed/test_level0_55_e%d_labels.bp'%e)
    bp.pack_ndarray_file(test_fnames, sift_dir+'/preprocessed/test_level0_55_e%d_fnames.bp'%e, chunk_size=255*4000)
    
    bp.pack_ndarray_file(train_dist_mat, sift_dir+'/preprocessed/train_level0_55_e%d_spm_dist_mat.bp'%e)
    

    sys.stderr.write('Experiment #%d Time: %f seconds\n' % (e, (time.time()-t)))

('BackG', 5105) Train:  250 Test:  250
('5N', 1440) Train:  250 Test:  250
('7n', 3444) Train:  250 Test:  250
('7N', 2579) Train:  250 Test:  250
('12N', 1230) Train:  250 Test:  250
('Pn', 3042) Train:  250 Test:  250
('VLL', 1287) Train:  250 Test:  250
('6N', 154) Train:  77 Test:  77
('Amb', 346) Train:  173 Test:  173
('R', 1082) Train:  250 Test:  250
('Tz', 1387) Train:  250 Test:  250
('RtTg', 2639) Train:  250 Test:  250
('LRt', 1050) Train:  250 Test:  250
('LC', 481) Train:  240 Test:  241
('AP', 483) Train:  241 Test:  242
('sp5', 3240) Train:  250 Test:  250
(3731, 200)
(3731,)
(3731,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0187_0032.png
(3733, 200)
(3733,)
(3733,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0228_0000.png


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:77: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:80: RuntimeWarning: invalid value encountered in divide


Train kernel shape:  (3731, 3731)
('BackG', 5105) Train:  250 Test:  250
('5N', 1440) Train:  250 Test:  250
('7n', 3444) Train:  250 Test:  250
('7N', 2579) Train:  250 Test:  250
('12N', 1230) Train:  250 Test:  250
('Pn', 3042) Train:  250 Test:  250
('VLL', 1287) Train:  250 Test:  250
('6N', 154) Train:  77 Test:  77
('Amb', 346) Train:  173 Test:  173
('R', 1082) Train:  250 Test:  250
('Tz', 1387) Train:  250 Test:  250
('RtTg', 2639) Train:  250 Test:  250
('LRt', 1050) Train:  250 Test:  250
('LC', 481) Train:  240 Test:  241
('AP', 483) Train:  241 Test:  242
('sp5', 3240) Train:  250 Test:  250
(3731, 200)
(3731,)
(3731,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0214_0022.png
(3733, 200)
(3733,)
(3733,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0299_0020.png


Experiment #0 Time: 2.616517 seconds


Train kernel shape:  (3731, 3731)
('BackG', 5105) Train:  250 Test:  250
('5N', 1440) Train:  250 Test:  250
('7n', 3444) Train:  250 Test:  250
('7N', 2579) Train:  250 Test:  250
('12N', 1230) Train:  250 Test:  250
('Pn', 3042) Train:  250 Test:  250
('VLL', 1287) Train:  250 Test:  250
('6N', 154) Train:  77 Test:  77
('Amb', 346) Train:  173 Test:  173
('R', 1082) Train:  250 Test:  250
('Tz', 1387) Train:  250 Test:  250
('RtTg', 2639) Train:  250 Test:  250
('LRt', 1050) Train:  250 Test:  250
('LC', 481) Train:  240 Test:  241
('AP', 483) Train:  241 Test:  242
('sp5', 3240) Train:  250 Test:  250
(3731, 200)
(3731,)
(3731,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0185_0014.png
(3733, 200)
(3733,)
(3733,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0217_0024.png


...

Experiment #7 Time: 2.630759 seconds


Train kernel shape:  (3731, 3731)
('BackG', 5105) Train:  250 Test:  250
('5N', 1440) Train:  250 Test:  250
('7n', 3444) Train:  250 Test:  250
('7N', 2579) Train:  250 Test:  250
('12N', 1230) Train:  250 Test:  250
('Pn', 3042) Train:  250 Test:  250
('VLL', 1287) Train:  250 Test:  250
('6N', 154) Train:  77 Test:  77
('Amb', 346) Train:  173 Test:  173
('R', 1082) Train:  250 Test:  250
('Tz', 1387) Train:  250 Test:  250
('RtTg', 2639) Train:  250 Test:  250
('LRt', 1050) Train:  250 Test:  250
('LC', 481) Train:  240 Test:  241
('AP', 483) Train:  241 Test:  242
('sp5', 3240) Train:  250 Test:  250
(3731, 200)
(3731,)
(3731,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0218_0020.png
(3733, 200)
(3733,)
(3733,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0301_0008.png


Experiment #8 Time: 2.694012 seconds


Train kernel shape:  (3731, 3731)


Experiment #9 Time: 2.699769 seconds


In [5]:
# build training data

# sift_dir = '/oasis/projects/nsf/csd395/yuncong/Brain/learning/sift'
# sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift'
sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

def compute_intersection_kernel_oneJob(i, j1):
#     dist = .5 * np.sum(train_data_normalized + h - np.abs(train_data_normalized - h), axis=1)
    dist = np.minimum(train_data_normalized[i], train_data_normalized[j1:]).sum(axis=1)
    return dist

exprmt = 10

for e in xrange(exprmt):
    
    t = time.time()

    train_data = []
    train_labels = []
    train_fnames = []

    test_data = []
    test_labels = []
    test_fnames = []

    n_sample = 500
    for name in labels:

        hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
        hists1 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l1.bp' % {'name': name})
        hists2 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l2.bp' % {'name': name})
        files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})
    #     print (hists0.shape, hists1.shape, hists2.shape, files.shape)

        n_total = hists0.shape[0]
        print (name, n_total),

        random_indices = np.random.choice(range(n_total), min(n_total, n_sample), replace=False)

        frac = int(0.6*len(random_indices))
        random_train = random_indices[:frac]
        random_test = random_indices[frac:]
        n_train = len(random_train)
        n_test = len(random_test)

    #     train_hists = np.c_[train_hists0[random_indices], 
    #                         train_hists1[random_indices].reshape((n_train, -1)), 
    #                         train_hists2[random_indices].reshape((n_train, -1))]
        train_hists = np.c_[hists0[random_train], 
                            hists1[random_train].reshape((n_train, -1)), 
                            hists2[random_train].reshape((n_train, -1))]
        test_hists =  np.c_[hists0[random_test], 
                            hists1[random_test].reshape((n_test, -1)), 
                            hists2[random_test].reshape((n_test, -1))]

    #     train_files = train_files_[random_indices].reshape((n_train,))
        train_files = files[random_train].reshape((n_train,))
        test_files = files[random_test].reshape((n_test,))

        train_data.append(train_hists)
        train_labels.append(np.ones((n_train, )) * labels_index[name])
        train_fnames.append(train_files)

        test_data.append(test_hists)
        test_labels.append(np.ones((n_test, )) * labels_index[name])
        test_fnames.append(test_files)

        print 'Train: ', n_train, 
        print 'Test: ', n_test

    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)
    train_fnames = np.concatenate(train_fnames)
    n_train = train_data.shape[0]

    test_data = np.concatenate(test_data)
    test_labels = np.concatenate(test_labels)
    test_fnames = np.concatenate(test_fnames)
    n_test = test_data.shape[0]
    
    print train_data.shape
    print train_labels.shape
    print train_fnames.shape, train_fnames.dtype
    print train_fnames[0]

    print test_data.shape
    print test_labels.shape
    print test_fnames.shape, test_fnames.dtype
    print test_fnames[0]
    
    train_data_normalized = train_data / train_data.sum(axis=1)[:,None].astype(np.float)
    train_data_normalized = np.nan_to_num(train_data_normalized)

    test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)
    test_data_normalized = np.nan_to_num(test_data_normalized)
    
    train_dist_triangle = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, i) 
                                         for i in range(n_train)))

    train_dist_mat = np.empty((n_train, n_train))
    train_dist_mat[np.triu_indices(n_train)] = train_dist_triangle
    r = np.tril_indices(n_train)
    train_dist_mat[r] = train_dist_mat.T[r]
    print 'Train kernel shape: ', train_dist_mat.shape
    
    bp.pack_ndarray_file(train_data_normalized, sift_dir+'/preprocessed/train_level2_64_e%d_data.bp'%e)
    bp.pack_ndarray_file(train_labels, sift_dir+'/preprocessed/train_level2_64_e%d_labels.bp'%e)
    bp.pack_ndarray_file(train_fnames, sift_dir+'/preprocessed/train_level2_64_e%d_fnames.bp'%e, chunk_size=255*4000)

    bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/test_level2_64_e%d_data.bp'%e)
    bp.pack_ndarray_file(test_labels, sift_dir+'/preprocessed/test_level2_64_e%d_labels.bp'%e)
    bp.pack_ndarray_file(test_fnames, sift_dir+'/preprocessed/test_level2_64_e%d_fnames.bp'%e, chunk_size=255*4000)
    
    bp.pack_ndarray_file(train_dist_mat, sift_dir+'/preprocessed/train_level2_64_e%d_spm_dist_mat.bp'%e)
    

    sys.stderr.write('Experiment #%d Time: %f seconds\n' % (e, (time.time()-t)))

('BackG', 5105) Train:  300 Test:  200
('5N', 1440) Train:  300 Test:  200
('7n', 3444) Train:  300 Test:  200
('7N', 2579) Train:  300 Test:  200
('12N', 1230) Train:  300 Test:  200
('Pn', 3042) Train:  300 Test:  200
('VLL', 1287) Train:  300 Test:  200
('6N', 154) Train:  92 Test:  62
('Amb', 346) Train:  207 Test:  139
('R', 1082) Train:  300 Test:  200
('Tz', 1387) Train:  300 Test:  200
('RtTg', 2639) Train:  300 Test:  200
('LRt', 1050) Train:  300 Test:  200
('LC', 481) Train:  288 Test:  193
('AP', 483) Train:  289 Test:  194
('sp5', 3240) Train:  300 Test:  200
(4476, 4200)

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:91: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:94: RuntimeWarning: invalid value encountered in divide



(4476,)
(4476,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0208_0006.png
(2988, 4200)
(2988,)
(2988,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0305_0018.png
Train kernel shape:  (4476, 4476)
('BackG', 5105) Train:  300 Test:  200
('5N', 1440) Train:  300 Test:  200
('7n', 3444) Train:  300 Test:  200
('7N', 2579) Train:  300 Test:  200
('12N', 1230) Train:  300 Test:  200
('Pn', 3042) Train:  300 Test:  200
('VLL', 1287) Train:  300 Test:  200
('6N', 154) Train:  92 Test:  62
('Amb', 346) Train:  207 Test:  139
('R', 1082) Train:  300 Test:  200
('Tz', 1387) Train:  300 Test:  200
('RtTg', 2639) Train:  300 Test:  200
('LRt', 1050) Train:  300 Test:  200
('LC', 481) Train:  288 Test:  193
('AP', 483) Train:  289 Test:  194
('sp5', 3240) Train:  300 Test:  200
(4476, 4200)
(4476,)
(4476,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0293_0033.png
(29

Experiment #0 Time: 31.288881 seconds


Train kernel shape:  (4476, 4476)
('BackG', 5105) Train:  300 Test:  200
('5N', 1440) Train:  300 Test:  200
('7n', 3444) Train:  300 Test:  200
('7N', 2579) Train:  300 Test:  200
('12N', 1230) Train:  300 Test:  200
('Pn', 3042) Train:  300 Test:  200
('VLL', 1287) Train:  300 Test:  200
('6N', 154) Train:  92 Test:  62
('Amb', 346) Train:  207 Test:  139
('R', 1082) Train:  300 Test:  200
('Tz', 1387) Train:  300 Test:  200
('RtTg', 2639) Train:  300 Test:  200
('LRt', 1050) Train:  300 Test:  200
('LC', 481) Train:  288 Test:  193
('AP', 483) Train:  289 Test:  194
('sp5', 3240) Train:  300 Test:  200
(4476, 4200)
(4476,)
(4476,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0155_0018.png
(2988, 4200)
(2988,)
(2988,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0164_0014.png


...

Experiment #7 Time: 28.909425 seconds


Train kernel shape:  (4476, 4476)
('BackG', 5105) Train:  300 Test:  200
('5N', 1440) Train:  300 Test:  200
('7n', 3444) Train:  300 Test:  200
('7N', 2579) Train:  300 Test:  200
('12N', 1230) Train:  300 Test:  200
('Pn', 3042) Train:  300 Test:  200
('VLL', 1287) Train:  300 Test:  200
('6N', 154) Train:  92 Test:  62
('Amb', 346) Train:  207 Test:  139
('R', 1082) Train:  300 Test:  200
('Tz', 1387) Train:  300 Test:  200
('RtTg', 2639) Train:  300 Test:  200
('LRt', 1050) Train:  300 Test:  200
('LC', 481) Train:  288 Test:  193
('AP', 483) Train:  289 Test:  194
('sp5', 3240) Train:  300 Test:  200
(4476, 4200)
(4476,)
(4476,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0306_0017.png
(2988, 4200)
(2988,)
(2988,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_BackG_0295_0005.png


Experiment #8 Time: 36.326741 seconds


Train kernel shape:  (4476, 4476)


Experiment #9 Time: 27.623106 seconds
